<a href="https://colab.research.google.com/github/HARSHIT097/Learnings/blob/main/Play_with_tsdata001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import datetime
import random
import numpy as np

# Bike ride sharing data

Create Bike Rides Data

In [ ]:
random.seed(38)
ride_start_time = datetime.datetime.today()
ride_end_time = ride_start_time + datetime.timedelta(minutes=20)
ride_start_times = []
ride_end_times = []
count_of_rides = 6

for ride in range(count_of_rides):
    minutes_between_rides = random.randint(8, 100)
    ride_start_time = ride_end_time + datetime.timedelta(minutes=minutes_between_rides)
    
    minutes_riding = random.randint(5, 25)
    ride_end_time = ride_start_time + datetime.timedelta(minutes=minutes_riding)
    
    ride_start_times.append(ride_start_time)
    ride_end_times.append(ride_end_time)

In [ ]:
data = {'ride_start_time': ride_start_times,
        'ride_end_time':  ride_end_times
       }
df_bike_rides = pd.DataFrame(data)

In [ ]:
df_bike_rides.head(2)

,ride_start_time,ride_end_time
0,2021-05-14 23:28:59.025882,2021-05-14 23:46:59.025882
1,2021-05-15 00:48:59.025882,2021-05-15 00:56:59.025882


In [ ]:
#Find the Mean Duration of Time, in Minutes, Bike is Left Idle Between Rides

In [ ]:
df_bike_rides['previous_ride_end_time'] = df_bike_rides['ride_end_time'].shift(periods=1)

In [ ]:
df_bike_rides['duration_bike_idle_between_rides'] = df_bike_rides['ride_start_time'] - df_bike_rides['previous_ride_end_time']

In [ ]:
df_bike_rides['minutes_bike_idle_between_rides'] = df_bike_rides['duration_bike_idle_between_rides'].dt.total_seconds()/60

In [ ]:
df_bike_rides.tail(2)

,ride_start_time,ride_end_time,previous_ride_end_time,duration_bike_idle_between_rides,minutes_bike_idle_between_rides
4,2021-05-15 04:19:59.025882,2021-05-15 04:25:59.025882,2021-05-15 03:24:59.025882,0 days 00:55:00,55.0
5,2021-05-15 05:50:59.025882,2021-05-15 06:00:59.025882,2021-05-15 04:25:59.025882,0 days 01:25:00,85.0


In [ ]:
#calculate the mean minutes_bike_idle_between_rides 
avg_minutes_bikes_idle_between_rides = df_bike_rides['minutes_bike_idle_between_rides'].mean()
avg_minutes_bikes_idle_between_rides

63.0

Duration Idle Time Between Bike Rides Per Unique Bike¶

In [ ]:
#create a pandas dataframe with details on bike ride times, the bike id and the start and end station.
data = {'ride_start_time': ride_start_times,
        'ride_end_time':  ride_end_times,
        'bike_id': [1, 22, 1, 1, 22, 22],
        'start_station': ["21st & Folsom", "21st & Folsom", "4th & King", "24th & Valencia", "4th & King",  "16th and Mission"],
        'end_station':  ["4th & King", "4th & King", "24th & Valencia", "Embarcadero & Market",  "16th and Mission", "4th & King"]
       }
df_bike_sharing = pd.DataFrame(data)

In [ ]:
#sort df_bike_sharing first by the bike_id column and then the ride_start_time column.
df_bike_sharing.sort_values(by=['bike_id', 'ride_start_time'], inplace=True)

In [ ]:
df_bike_sharing.head()

,ride_start_time,ride_end_time,bike_id,start_station,end_station
0,2021-05-14 23:28:59.025882,2021-05-14 23:46:59.025882,1,21st & Folsom,4th & King
2,2021-05-15 01:12:59.025882,2021-05-15 01:28:59.025882,1,4th & King,24th & Valencia
3,2021-05-15 03:05:59.025882,2021-05-15 03:24:59.025882,1,24th & Valencia,Embarcadero & Market
1,2021-05-15 00:48:59.025882,2021-05-15 00:56:59.025882,22,21st & Folsom,4th & King
4,2021-05-15 04:19:59.025882,2021-05-15 04:25:59.025882,22,4th & King,16th and Mission


For the bike_id column, I shift down values by 1 to create a new column called previous_bike_id. I do this so I can easily compare a bike id to the previous ride's ID to identify the last ride by a bike id for a day.

Similar to LAG()

In [ ]:
df_bike_sharing['previous_bike_id'] = df_bike_sharing['bike_id'].shift(periods=1)

SELECT *, LAG(bike_id) OVER (ORDER BY bike_id ASC, ride_start_time ASC) AS previous_bike_id
FROM df_bike_sharing 
ORDER BY bike_id ASC, ride_start_time ASC

In [ ]:
df_bike_sharing['previous_ride_end_time'] = df_bike_sharing['ride_end_time'].shift(periods=1)

In [ ]:
# to calculate a duration the bike was idle in a row if it's a comparison for the same bike id. Below I show the code to replace all values 
#in a row with NaN or NaT if the condition is false.
df_bike_sharing.where(df_bike_sharing['bike_id'] == df_bike_sharing['previous_bike_id'])

,ride_start_time,ride_end_time,bike_id,start_station,end_station,previous_bike_id,previous_ride_end_time
0,NaT,NaT,NaN,NaN,NaN,NaN,NaT
2,2021-05-15 01:12:59.025882,2021-05-15 01:28:59.025882,1.0,4th & King,24th & Valencia,1.0,2021-05-14 23:46:59.025882
3,2021-05-15 03:05:59.025882,2021-05-15 03:24:59.025882,1.0,24th & Valencia,Embarcadero & Market,1.0,2021-05-15 01:28:59.025882
1,NaT,NaT,NaN,NaN,NaN,NaN,NaT
4,2021-05-15 04:19:59.025882,2021-05-15 04:25:59.025882,22.0,4th & King,16th and Mission,22.0,2021-05-15 00:56:59.025882
5,2021-05-15 05:50:59.025882,2021-05-15 06:00:59.025882,22.0,16th and Mission,4th & King,22.0,2021-05-15 04:25:59.025882


In [ ]:
df_bike_sharing['duration_bike_idle'] = (df_bike_sharing['ride_start_time']-df_bike_sharing['previous_ride_end_time']).where(df_bike_sharing['bike_id'] == df_bike_sharing['previous_bike_id'])

In [ ]:
#I drop previous_bike_id and previous_ride_end_time since they were intermediary outputs used for calculations.
# They're not necessary for a final presentation of the critical details.
df_bike_sharing['duration_bike_idle_seconds'] = df_bike_sharing['duration_bike_idle'].dt.total_seconds()

Here is the average seconds each bike_id is idle during the day in seconds between the first and last ride. I group by the bike_id column and calculate the mean of the duration_bike_idle_seconds_time column. I reset the index and rename the columns so this final output is easier to understand.

Bike id of 22 was left idle longer between rides than the bike id of 1.


In [ ]:
df_bike_sharing.groupby('bike_id')['duration_bike_idle_seconds'].mean().reset_index().rename(columns={'duration_bike_idle_seconds': 'avg_seconds_idle_between_rides'})

,bike_id,avg_seconds_idle_between_rides
0,1,5490.0
1,22,8640.0
